In [58]:
import tensorflow as tf
import keras
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import numpy as np
from tkinter import filedialog, messagebox


In [59]:
train_dir = r"C:\Users\Johin\Desktop\Practice\Melanoma Skin Cancer\melanoma_cancer_dataset\train"
test_dir = r"C:\Users\Johin\Desktop\Practice\Melanoma Skin Cancer\melanoma_cancer_dataset\test"

In [60]:
train_gen = ImageDataGenerator(rescale = 1./255,
                               horizontal_flip = True,
                               rotation_range = 40,
                               shear_range=0.2,
                               width_shift_range = 0.2,
                               height_shift_range = 0.2,
                               zoom_range = 0.2,
                               fill_mode = 'nearest')
test_gen = ImageDataGenerator(rescale = 1./255)

In [61]:
train = train_gen.flow_from_directory(train_dir,
                                      target_size = (150,150),
                                      class_mode = 'binary',
                                      batch_size = 100
                                     )
test = test_gen.flow_from_directory(test_dir,
                                   target_size = (150,150),
                                   batch_size = 100,
                                   class_mode = 'binary')

Found 9605 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [62]:
model = keras.Sequential([keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape= (150,150,3)),
                         keras.layers.MaxPooling2D(2,2),
                         keras.layers.Conv2D(64, (3,3), activation = 'relu'),
                         keras.layers.MaxPooling2D(2,2),
                         keras.layers.Flatten(),
                         keras.layers.Dense(128, activation = 'relu'),
                         keras.layers.Dense(1, activation = 'sigmoid')])
model.compile(optimizer = RMSprop(learning_rate = 0.001), loss = 'binary_crossentropy',metrics = ['accuracy'])

In [64]:
model.fit(train,
         steps_per_epoch = 10,
         epochs = 50,
         verbose = 1)

Epoch 1/50
10/10 [==============================] - 40s 4s/step - loss: 4.2131 - accuracy: 0.5480
Epoch 2/50
10/10 [==============================] - 36s 4s/step - loss: 0.6680 - accuracy: 0.6070
Epoch 3/50
10/10 [==============================] - 32s 3s/step - loss: 0.6698 - accuracy: 0.6420
Epoch 4/50
10/10 [==============================] - 32s 3s/step - loss: 0.5938 - accuracy: 0.6700
Epoch 5/50
10/10 [==============================] - 28s 3s/step - loss: 0.5914 - accuracy: 0.6685
Epoch 6/50
10/10 [==============================] - 31s 3s/step - loss: 0.5717 - accuracy: 0.7180
Epoch 7/50
10/10 [==============================] - 30s 3s/step - loss: 0.5324 - accuracy: 0.7250
Epoch 8/50
10/10 [==============================] - 28s 3s/step - loss: 0.5585 - accuracy: 0.6950
Epoch 9/50
10/10 [==============================] - 30s 3s/step - loss: 0.5286 - accuracy: 0.7280
Epoch 10/50
10/10 [==============================] - 29s 3s/step - loss: 0.4780 - accuracy: 0.7680
Epoch 11/50
10/10 [

In [65]:
model.evaluate(test)

10/10 [==============================] - 12s 1s/step - loss: 0.2744 - accuracy: 0.8840


[0.27444493770599365, 0.8840000033378601]

In [ ]:
model.save("melanoma_model.h5")

In [ ]:
melanoma_model = model

root = tk.Tk()
root.title("Skin Cancer Detector")
root.geometry("400x400")

image_label = tk.Label(root)
image_label.pack(pady=10)

prediction_label = tk.Label(root, font=('Helvetica', 18))
prediction_label.pack(pady=10)

def open_file_dialog():
    filenames = filedialog.askopenfilenames(initialdir="/", title="Select an Image",
                                          filetypes=[("Image Files", "*.png *.jpg *.jpeg")])
    if filenames:
        for filename in filenames:
            image = Image.open(filename)
            image = image.resize((150, 150))
            image = np.array(image)
            if image.shape != (150, 150, 3):
                messagebox.showerror("Error", "The selected image is not of the correct size or shape (150x150x3).")
                return
            image = image.reshape(1, 150, 150, 3)
            result = melanoma_model.predict(image)
            if result > 0.5:
                messagebox.showinfo("Prediction", "The selected image is classified as melanoma.")
            else:
                messagebox.showinfo("Prediction", "The selected image is classified as not melanoma.")

open_file_button = tk.Button(root, text="Open file", command=open_file_dialog)
open_file_button.pack(pady=10)

root.mainloop()
